In [1]:
import numpy as np
from oceantracker import main
datasource= 1
duration_days= 1

if datasource==1:
    output_file_base= 'Sounds'
    input_dir =  'G:\\Hindcasts_large\\MalbroughSounds_10year_benPhD\\2008'
    file_mask  = 'schism_marl200801*.nc'
    root_output_dir = 'F:\\OceanTrackerOuput\\OceanTrackerProfiling'
    time_step = 600  #10min
    release_interval = 3600
    pulse_size = 200
    calculation_interval = 3 * 3600


points= [[1595000, 5482600. , -1],
            [1599000, 5486200, -1],
            [1597682.1237, 5489972.7479, -1],
            [1598604.1667, 5490275.5488, -1],
            [1598886.4247, 5489464.0424, -1],
            [1597917.3387, 5489000, -1]]
poly_points = [[1597682.1237, 5489972.7479],
                [1598604.1667, 5490275.5488],
                [1598886.4247, 5489464.0424],
                [1597917.3387, 5489000],
                [1597300, 5489000], [1597682.1237, 5489972.7479]]



params = \
    {'root_output_dir': root_output_dir, 
    'output_file_base': output_file_base, 
    'max_duration' :duration_days *24*3600,
    'debug': True, 
    'time_step': time_step,
    'screen_output_time_interval':time_step,
    'reader': {'class_name': 'oceantracker.reader.schism_reader.SCHSIMreaderNCDF',
                'input_dir': input_dir,
                'file_mask': file_mask,
                'field_variables': {'water_temperature': 'temp'}
                },
    'write_tracks': False,
    'dispersion': {'A_H': .2, 'A_V': 0.001},
    'particle_release_groups': [{'points': points,
                                    'pulse_size': pulse_size, 'release_interval': release_interval,
                                    'allow_release_in_dry_cells': True},
                                    {'class_name': 'oceantracker.particle_release_groups.polygon_release.PolygonRelease',
                                    'points': poly_points,
                                    'pulse_size': pulse_size, 'release_interval': release_interval}
                                    ],
    'particle_properties': [{'class_name': 'oceantracker.particle_properties.age_decay.AgeDecay',
                                'decay_time_scale': 1. * 3600 * 24}],
    'event_loggers': [{'class_name': 'oceantracker.event_loggers.log_polygon_entry_and_exit.LogPolygonEntryAndExit',
                                    'particle_prop_to_write_list': ['ID', 'x', 'IDrelease_group', 'status', 'age'],
                                    'polygon_list': [{'user_polygon_name': 'A', 'points': (np.asarray(poly_points) + np.asarray([-5000, 0])).tolist()},
                                                                                                ]
                                    }],
    'trajectory_modifiers': [{'class_name': 'oceantracker.resuspension.BasicResuspension',
                                            'critical_friction_velocity': .001}],
    'velocity_modifiers': [{'class_name': 'oceantracker.velocity_modifiers.terminal_velocity.TerminalVelocity', 'variance': 0.001}],
    'fields':  [{'class_name': 'oceantracker.fields.friction_velocity.FrictionVelocity'}],
    'particle_statistics' : [
                        {'class_name': 'oceantracker.particle_statistics.gridded_statistics.GriddedStats2D_agedBased',
                            'calculation_interval': calculation_interval, 'particle_property_list': ['water_depth'],
                            'grid_size': [220, 221],
                            'min_age_to_bin': 0., 'max_age_to_bin': 3. * 24 * 3600, 'age_bin_size': 3600.},
                        {'class_name': 'oceantracker.particle_statistics.polygon_statistics.PolygonStats2D_ageBased',
                            'calculation_interval': calculation_interval, 'particle_property_list': ['water_depth'],
                            'min_age_to_bin': 0., 'max_age_to_bin': 3. * 24 * 3600, 'age_bin_size': 3600.,
                            'polygon_list': [{'points': poly_points}]}  ]
                }
        
